In [2]:
#  Importing necessary libraries

import numpy as np
import pandas as pd
import os 
import requests
from concurrent.futures import ThreadPoolExecutor
import cv2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from glob import glob

In [7]:
def get_logo(domain):
    return f"https://logo.clearbit.com/{domain}"

df = pd.read_parquet("logos.snappy.parquet")

df["logo"] = df["domain"].apply(get_logo)

domains = df["domain"].unique()

df.head(15)

,domain,logo
0,stanbicbank.co.zw,https://logo.clearbit.com/stanbicbank.co.zw
1,astrazeneca.ua,https://logo.clearbit.com/astrazeneca.ua
2,autosecuritas-ct-seysses.fr,https://logo.clearbit.com/autosecuritas-ct-sey...
3,ovb.ro,https://logo.clearbit.com/ovb.ro
4,mazda-autohaus-hellwig-hoyerswerda.de,https://logo.clearbit.com/mazda-autohaus-hellw...
5,toyota-buchreiter-eisenstadt.at,https://logo.clearbit.com/toyota-buchreiter-ei...
6,ebay.cn,https://logo.clearbit.com/ebay.cn
7,greatplacetowork.com.bo,https://logo.clearbit.com/greatplacetowork.com.bo
8,wurth-international.com,https://logo.clearbit.com/wurth-international.com
9,plameco-hannover.de,https://logo.clearbit.com/plameco-hannover.de


In [ ]:
os.makedirs("logos", exist_ok=True)
from requests.exceptions import RequestException 

successful_downloads = 0
failed_downloads = 0

def download_logo(domain):
    logo_url = f"https://logo.clearbit.com/{domain}"
    
    try:
        response = requests.get(logo_url, timeout=5)  # Add timeout
        
        if response.status_code == 200:
            file_path = f"logos/{domain}.png"
            with open(file_path, "wb") as file:
                file.write(response.content)
            print(f"✅ Downloaded: {domain}")
            return 1  # Success
        else:
            print(f"❌ Logo not found: {domain}")
            return 0  # Failure
            
    except RequestException as e:
        print(f"🚨 Network error for {domain}: {e}")
        return 0  # Failure

# Use ThreadPoolExecutor and collect results
with ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(download_logo, domains))

# Count successes and failures
successful_downloads = sum(results)
failed_downloads = len(results) - successful_downloads

print(f"\nSummary:")
print(f"✅ Successful downloads: {successful_downloads}")
print(f"❌ Failed downloads: {failed_downloads}")

In [ ]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = preprocess_input(img)
    return img

def extract_features(image_paths):
    features = []
    for img_path in image_paths:
        img = load_and_preprocess_image(img_path)
        img = np.expand_dims(img, axis=0)
        feature = model.predict(img)
        feature = feature.flatten()
        features.append(feature)
    return np.array(features)

base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

image_paths = glob("logos/*.png")  
features = extract_features(image_paths)

clusters = [3, 5, 10, 20, 50, 70, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

os.makedirs(f"clusters", exist_ok=True)

for num_clusters in clusters:
    print(f"Running experiment with {num_clusters} clusters")

    kmeans = KMeans(n_clusters=num_clusters, random_state=42)lo
    kmeans.fit(features)
    labels = kmeans.labels_

    current_experiment_dir = os.path.join("clusters", f"experiment_{num_clusters}")

    for cluster in range(num_clusters):
        cluster_dir = os.path.join(current_experiment_dir, f"cluster_{cluster}")
        os.makedirs(cluster_dir, exist_ok=True)
        
    for img_path, label in zip(image_paths, labels):
        img_name = os.path.basename(img_path)
        dest_path = os.path.join(current_experiment_dir, f"cluster_{label}", img_name)
        cv2.imwrite(dest_path, cv2.imread(img_path))

    for cluster in range(num_clusters):
        plt.figure(figsize=(10, 5))
        plt.suptitle(f'Cluster {cluster}')
        cluster_images = glob(os.path.join(current_experiment_dir, f"cluster_{cluster}", "*.png"))